In [1]:
# import necessary libraries
import pandas as pd
import os
import glob


In [98]:
all_df = pd.read_csv("./data/csv/final_csv_df/merged_files/All_merged_2017_2021.csv")
all_df = all_df.sort_values('year_quarter').reset_index(drop=True)


/home/mshoush/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Registration_number,Name_x,Type,Registered_for_VAT,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,Name_y,Business_area_by_EMTAK_system_letter,Business_are_by_EMTAK_system_number,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date


In [99]:
before_df = pd.read_csv("./data/csv/final_csv_df/merged_files/Before_Covid_merged.csv")
before_df = before_df.sort_values('year_quarter').reset_index(drop=True)


,Registration_number,Name_x,Type,Registered_for_VAT,Business_area_by_EMTAK_system,County,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,Name_y,Business_area_by_EMTAK_system_letter,Business_are_by_EMTAK_system_number,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date


In [100]:
During_df = pd.read_csv("./data/csv/final_csv_df/merged_files/During_Covid_merged.csv")
During_df = During_df.sort_values('year_quarter').reset_index(drop=True)


,Registration_number,Name_x,Type,Registered_for_VAT,Business_area_by_EMTAK_system,County,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,Name_y,Business_area_by_EMTAK_system_letter,Business_are_by_EMTAK_system_number,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date


In [153]:
def clean_data(df):
    all_df = df
    # drop turnover missing values
    all_df = all_df[all_df['Turnover'].notna()]
    
    # Remove all business types except äriühing
    all_df = all_df[all_df['Type']=="Äriühing"]
    
    # Drop duplicates
    all_df = all_df.drop_duplicates()

    # impute missing values 
    all_df['County'] = all_df['County'].replace(np.nan, "other")
    all_df['Business_area_by_EMTAK_system'] = all_df['Business_area_by_EMTAK_system'].fillna("other")
    
    # same as number of employee, will do it by grouping during feat eng
    #all_df['All_taxes'] = all_df['All_taxes'].fillna(all_df['All_taxes'].mean())
    #all_df['Labor_taxes'] = all_df['Labor_taxes'].fillna(all_df['Labor_taxes'].mean())
    
    # will add this to the feat engineering
    # to do it by grouping
    #all_df['number_of_employees'] = all_df['number_of_employees'].fillna(int(all_df['number_of_employees'].mean()))

    # nan values mean companu still exist
    all_df['Deletion_date']=all_df['Deletion_date'].fillna(int(1))
    # Other values mean company died
    all_df.loc[(all_df['Deletion_date'] !=1), 'Deletion_date'] = 0
    
    #replacing negative turover with small nr 
    all_df['Turnover'] = all_df['Turnover'].apply(lambda x : x if x > 0 else 0.000001)

    #dropping name_y
    all_df.drop(columns=['Name_y'], inplace=True)
    
    return all_df




In [154]:
all_cl = clean_data(all_df).reset_index(drop=True)
all_cl

,Registration_number,Name_x,Type,Registered_for_VAT,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,Business_area_by_EMTAK_system_letter,Business_are_by_EMTAK_system_number,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date
0,12166929,ILUKLUBI OÜ,Äriühing,jah,Pärnu maakond,MUUD TEENINDAVAD TEGEVUSED,1805.75,1321.56,5377.640000,3.0,2017_i,S,9602,S11002-S11003,8,2011-09-26,1
1,11189667,"PIIRILINN, OÜ",Äriühing,jah,Ida-Viru maakond,EHITUS,272.00,0.00,0.000001,NaN,2017_i,F,4110,S11002-S11003,8,2006-01-25,1
2,12166881,STANDARTINVEST OÜ,Äriühing,jah,Harju maakond,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,205.02,180.00,59140.400000,1.0,2017_i,G,4639,S11002-S11003,8,2011-09-26,1
3,11189673,"VIRUMAA TEHNOPARK, OÜ",Äriühing,jah,Ida-Viru maakond,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",1501.40,0.00,0.000001,NaN,2017_i,M,7111,S11002-S11003,8,2006-01-25,1
4,12166852,"LAHEVIK, OÜ",Äriühing,jah,Harju maakond,TÖÖTLEV TÖÖSTUS,1664.56,238.52,35216.000000,1.0,2017_i,C,3109,S11002-S11003,8,2011-09-26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281776,14974378,5K TRIP OÜ,Äriühing,jah,Harju ( Kose vald ),HALDUS- JA ABITEGEVUSED,106.47,0.00,1071.000000,NaN,2021_iii,N,7739,S11002-S11003,8,2020-05-19,1
281777,11927993,KEFI GRUPP OÜ,Äriühing,jah,Harju ( Tallinn ),HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,288.17,0.00,9605.400000,1.0,2021_iii,N,7810,S11002-S11003,8,2010-04-20,1
281778,11928069,INSTEK OÜ,Äriühing,jah,Harju ( Raasiku vald ),"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",4406.76,1132.86,19080.000000,1.0,2021_iii,L,6820,S11002-S11003,8,2010-04-20,1
281779,11935633,EPOLICE SOLUTIONS OÜ,Äriühing,jah,Lääne-Viru ( Vinni vald ),INFO JA SIDE,119.08,0.00,15703.790000,NaN,2021_iii,J,6202,S11002-S11003,8,2010-05-04,1


In [156]:
before_cl = clean_data(before_df).reset_index(drop=True)
before_cl

,Registration_number,Name_x,Type,Registered_for_VAT,Business_area_by_EMTAK_system,County,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,Business_area_by_EMTAK_system_letter,Business_are_by_EMTAK_system_number,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date
0,11498612,SIRGU OÜ,Äriühing,jah,TÖÖTLEV TÖÖSTUS,Tartu maakond,1734.04,886.13,6069.890000,1.0,2018_iii,C,1812,S11002-S11003,8,2008-06-27,1
1,11498641,ALTOPAL OÜ,Äriühing,jah,VEONDUS JA LAONDUS,Lääne-Viru maakond,6665.07,6452.08,180853.610000,6.0,2018_iii,H,4941,S11002-S11003,8,2008-06-27,1
2,12349877,KUMMELI MAJAD OÜ,Äriühing,ei,KINNISVARAALANE TEGEVUS,Harju maakond,250.00,0.00,1250.000000,NaN,2018_iii,L,6810,S11002-S11003,8,2012-09-21,1
3,11498693,AIMARK OÜ,Äriühing,jah,VEONDUS JA LAONDUS,Jõgeva maakond,NaN,NaN,0.000001,NaN,2018_iii,H,4941,S11002-S11003,8,2008-06-30,1
4,11498718,GROWLAND OÜ,Äriühing,jah,"PŐLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK",Jõgeva maakond,10521.48,5362.98,40371.480000,3.0,2018_iii,A,111,S11002-S11003,8,2008-06-30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134472,16076784,NEVETON OÜ,Äriühing,jah,EHITUS,Harju ( Anija vald ),11358.83,10589.10,46010.000000,4.0,2021_iii,N,8299,S11002-S11003,8,2020-10-15,1
134473,16076821,PROFFROOF.EE OÜ,Äriühing,jah,EHITUS,Harju ( Tallinn ),244.28,236.60,21132.710000,3.0,2021_iii,N,8299,S11002-S11003,8,2020-10-15,1
134474,16076838,EDIGITO OÜ,Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Harju ( Tallinn ),NaN,NaN,3737.440000,NaN,2021_iii,M,7311,S11002-S11003,8,2020-10-15,1
134475,16076844,DIGITAL NATION DATA OÜ,Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Harju ( Tallinn ),NaN,NaN,0.000001,NaN,2021_iii,M,7022,S11002-S11003,8,2020-10-15,1


In [158]:
during_cl = clean_data(During_df).reset_index(drop=True)
during_cl

,Registration_number,Name_x,Type,Registered_for_VAT,Business_area_by_EMTAK_system,County,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,Business_area_by_EMTAK_system_letter,Business_are_by_EMTAK_system_number,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date
0,10015770,"REPTEKS, OÜ",Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Lääne-Viru ( Kadrina vald ),0.00,0.00,5223.250000,NaN,2020_iv,M,7112,S11002-S11003,8,1992-06-18,1.0
1,14950082,MARJA EHITUS OÜ,Äriühing,jah,EHITUS,Pärnu ( Pärnu linn ),1829.46,2051.70,3686.000000,4.0,2020_iv,F,4391,S11002-S11003,8,2020-04-09,1.0
2,14950076,LIFTEST OMME OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),523.73,0.00,8891.860000,NaN,2020_iv,G,4520,S11002-S11003,8,2020-04-09,1.0
3,14950030,IGG TOOLING OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),1820.53,693.03,14569.640000,1.0,2020_iv,G,4675,S11002-S11003,8,2020-04-09,1.0
4,14950018,LERTISENTO OÜ,Äriühing,jah,HALDUS- JA ABITEGEVUSED,Harju ( Tallinn ),6282.95,4030.22,21159.220000,4.0,2020_iv,N,7820,S11002-S11003,8,2020-04-09,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31060,14991230,SEMPRE MEGLIO OÜ,Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Harju ( Tallinn ),NaN,NaN,121.160000,NaN,2021_iii,M,7022,S11002-S11003,8,2020-06-16,1.0
31061,14991247,ARSI GARAAZ OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Lääne-Viru ( Rakvere vald ),166.07,0.00,9317.770000,NaN,2021_iii,G,4520,S11002-S11003,8,2020-06-16,1.0
31062,14991276,REVFRESH MARKETPLACE OÜ,Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Lääne-Viru ( Rakvere linn ),440.00,0.00,2250.000000,NaN,2021_iii,M,7311,S11002-S11003,8,2020-06-16,1.0
31063,14991313,ESTONIAN NATURAL HONEY OÜ,Äriühing,jah,"PÕLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK",Pärnu ( Pärnu linn ),0.00,0.00,4993.910000,NaN,2021_iii,A,149,S11002-S11003,8,2020-06-16,1.0


In [160]:
all_cl.to_csv( "./data/csv/final_csv_df/cl_data/all.csv", index=False)
before_cl.to_csv( "./data/csv/final_csv_df/cl_data/before.csv", index=False)
during_cl.to_csv( "./data/csv/final_csv_df/cl_data/during.csv", index=False)

# Feat Engineering

In [161]:
all_cl_df = pd.read_csv("./data/csv/final_csv_df/cl_data/all.csv")
all_cl_df.head()

,Registration_number,Name_x,Type,Registered_for_VAT,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,Business_area_by_EMTAK_system_letter,Business_are_by_EMTAK_system_number,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date
0,12166929,ILUKLUBI OÜ,Äriühing,jah,Pärnu maakond,MUUD TEENINDAVAD TEGEVUSED,1805.75,1321.56,5377.640000,3.0,2017_i,S,9602,S11002-S11003,8,2011-09-26,1
1,11189667,"PIIRILINN, OÜ",Äriühing,jah,Ida-Viru maakond,EHITUS,272.00,0.00,0.000001,NaN,2017_i,F,4110,S11002-S11003,8,2006-01-25,1
2,12166881,STANDARTINVEST OÜ,Äriühing,jah,Harju maakond,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,205.02,180.00,59140.400000,1.0,2017_i,G,4639,S11002-S11003,8,2011-09-26,1
3,11189673,"VIRUMAA TEHNOPARK, OÜ",Äriühing,jah,Ida-Viru maakond,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",1501.40,0.00,0.000001,NaN,2017_i,M,7111,S11002-S11003,8,2006-01-25,1
4,12166852,"LAHEVIK, OÜ",Äriühing,jah,Harju maakond,TÖÖTLEV TÖÖSTUS,1664.56,238.52,35216.000000,1.0,2017_i,C,3109,S11002-S11003,8,2011-09-26,1


In [163]:
import numpy as np
import math
def feat_eng(g):
    ex_df=pd.DataFrame(g)    
    
    if ex_df['number_of_employees'].isnull().sum()>0:
      
        try:
            ex_df['number_of_employees'] = ex_df['number_of_employees'].fillna(int(ex_df['number_of_employees'].mean()))
        except:
            ex_df['number_of_employees'] = ex_df['number_of_employees'].fillna(int(1))
        
  
    
    if ex_df['Labor_taxes'].isnull().sum():
        #print("Labor_taxes")
        if math.isnan(ex_df['Labor_taxes'].mean()):
            ex_df['Labor_taxes'] = 0000.1
        else:
            ex_df['Labor_taxes'] = ex_df['Labor_taxes'].fillna(ex_df['Labor_taxes'].mean())
    
    
    if ex_df['All_taxes'].isnull().sum():
        if math.isnan(ex_df['All_taxes'].mean()):
            ex_df['All_taxes'] = 0000.1
        else:
            ex_df['All_taxes'] = ex_df['All_taxes'].fillna(ex_df['All_taxes'].mean())
           
        

    # Cumulative Turn over 
    ex_df['cum_sum_turnover'] = np.round(ex_df['Turnover'].cumsum(), 2)
    ex_df['cum_perc_turnover'] = np.round(100*ex_df['cum_sum_turnover']/ex_df['Turnover'].sum(), 2)

    # combined taxes
    ex_df['combined_taxes'] = np.round(ex_df['Labor_taxes'] + ex_df['All_taxes'], 2)

    # Cumulative Taxes
    ex_df['cum_sum_combined_taxes'] = np.round(ex_df['combined_taxes'].cumsum(), 2)
    #ex_df['cum_perc_combined_taxes'] = np.round(100*ex_df['cum_sum_combined_taxes']/ex_df['combined_taxes'].sum(), 2)

    # Revenue Growth
    ex_df['revenue_growth'] = np.round(ex_df.Turnover.pct_change(fill_method="ffill"),2)
    ex_df['revenue_growth'] = ex_df['revenue_growth'].fillna(0)


    # Profit per Employee
    ex_df["profit_per_employee"] = np.round(ex_df['Turnover']/ex_df['number_of_employees'], 2)


    return ex_df




In [164]:
all_feat_eng = all_cl_df.groupby("Registration_number").apply(feat_eng)
all_feat_eng

,Registration_number,Name_x,Type,Registered_for_VAT,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,...,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee
0,12166929,ILUKLUBI OÜ,Äriühing,jah,Pärnu maakond,MUUD TEENINDAVAD TEGEVUSED,1805.750,1321.56,5377.640000,3.0,...,S11002-S11003,8,2011-09-26,1,5377.64,16.61,3127.31,3127.31,0.00,1792.55
1,11189667,"PIIRILINN, OÜ",Äriühing,jah,Ida-Viru maakond,EHITUS,272.000,0.00,0.000001,1.0,...,S11002-S11003,8,2006-01-25,1,0.00,0.00,272.00,272.00,0.00,0.00
2,12166881,STANDARTINVEST OÜ,Äriühing,jah,Harju maakond,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,205.020,180.00,59140.400000,1.0,...,S11002-S11003,8,2011-09-26,1,59140.40,44.97,385.02,385.02,0.00,59140.40
3,11189673,"VIRUMAA TEHNOPARK, OÜ",Äriühing,jah,Ida-Viru maakond,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",1501.400,0.00,0.000001,1.0,...,S11002-S11003,8,2006-01-25,1,0.00,0.00,1501.40,1501.40,0.00,0.00
4,12166852,"LAHEVIK, OÜ",Äriühing,jah,Harju maakond,TÖÖTLEV TÖÖSTUS,1664.560,238.52,35216.000000,1.0,...,S11002-S11003,8,2011-09-26,1,35216.00,41.73,1903.08,1903.08,0.00,35216.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281776,14974378,5K TRIP OÜ,Äriühing,jah,Harju ( Kose vald ),HALDUS- JA ABITEGEVUSED,106.470,0.00,1071.000000,1.0,...,S11002-S11003,8,2020-05-19,1,3357.67,100.00,106.47,365.70,1.29,1071.00
281777,11927993,KEFI GRUPP OÜ,Äriühing,jah,Harju ( Tallinn ),HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,288.170,0.00,9605.400000,1.0,...,S11002-S11003,8,2010-04-20,1,61280.20,100.00,288.17,1906.93,-0.14,9605.40
281778,11928069,INSTEK OÜ,Äriühing,jah,Harju ( Raasiku vald ),"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",4406.760,1132.86,19080.000000,1.0,...,S11002-S11003,8,2010-04-20,1,38930.42,100.00,5539.62,23105.34,-0.04,19080.00
281779,11935633,EPOLICE SOLUTIONS OÜ,Äriühing,jah,Lääne-Viru ( Vinni vald ),INFO JA SIDE,119.080,0.00,15703.790000,1.0,...,S11002-S11003,8,2010-05-04,1,85797.21,100.00,119.08,6064.54,0.50,15703.79


In [167]:
before_cl_df = pd.read_csv("./data/csv/final_csv_df/cl_data/before.csv")
before_cl_df.head()

,Registration_number,Name_x,Type,Registered_for_VAT,Business_area_by_EMTAK_system,County,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,Business_area_by_EMTAK_system_letter,Business_are_by_EMTAK_system_number,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date
0,11498612,SIRGU OÜ,Äriühing,jah,TÖÖTLEV TÖÖSTUS,Tartu maakond,1734.04,886.13,6069.890000,1.0,2018_iii,C,1812,S11002-S11003,8,2008-06-27,1
1,11498641,ALTOPAL OÜ,Äriühing,jah,VEONDUS JA LAONDUS,Lääne-Viru maakond,6665.07,6452.08,180853.610000,6.0,2018_iii,H,4941,S11002-S11003,8,2008-06-27,1
2,12349877,KUMMELI MAJAD OÜ,Äriühing,ei,KINNISVARAALANE TEGEVUS,Harju maakond,250.00,0.00,1250.000000,NaN,2018_iii,L,6810,S11002-S11003,8,2012-09-21,1
3,11498693,AIMARK OÜ,Äriühing,jah,VEONDUS JA LAONDUS,Jõgeva maakond,NaN,NaN,0.000001,NaN,2018_iii,H,4941,S11002-S11003,8,2008-06-30,1
4,11498718,GROWLAND OÜ,Äriühing,jah,"PŐLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK",Jõgeva maakond,10521.48,5362.98,40371.480000,3.0,2018_iii,A,111,S11002-S11003,8,2008-06-30,1


In [169]:
before_feat_eng = before_cl_df.groupby("Registration_number").apply(feat_eng)
before_feat_eng

,Registration_number,Name_x,Type,Registered_for_VAT,Business_area_by_EMTAK_system,County,All_taxes,Labor_taxes,Turnover,number_of_employees,...,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee
0,11498612,SIRGU OÜ,Äriühing,jah,TÖÖTLEV TÖÖSTUS,Tartu maakond,1734.04,886.13,6069.890000,1.0,...,S11002-S11003,8,2008-06-27,1,6069.89,48.37,2620.17,2620.17,0.000000e+00,6069.89
1,11498641,ALTOPAL OÜ,Äriühing,jah,VEONDUS JA LAONDUS,Lääne-Viru maakond,6665.07,6452.08,180853.610000,6.0,...,S11002-S11003,8,2008-06-27,1,180853.61,53.66,13117.15,13117.15,0.000000e+00,30142.27
2,12349877,KUMMELI MAJAD OÜ,Äriühing,ei,KINNISVARAALANE TEGEVUS,Harju maakond,250.00,0.00,1250.000000,1.0,...,S11002-S11003,8,2012-09-21,1,1250.00,100.00,250.00,250.00,0.000000e+00,1250.00
3,11498693,AIMARK OÜ,Äriühing,jah,VEONDUS JA LAONDUS,Jõgeva maakond,0.10,0.10,0.000001,1.0,...,S11002-S11003,8,2008-06-30,1,0.00,0.00,0.20,0.20,0.000000e+00,0.00
4,11498718,GROWLAND OÜ,Äriühing,jah,"PŐLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK",Jõgeva maakond,10521.48,5362.98,40371.480000,3.0,...,S11002-S11003,8,2008-06-30,1,40371.48,29.36,15884.46,15884.46,0.000000e+00,13457.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134472,16076784,NEVETON OÜ,Äriühing,jah,EHITUS,Harju ( Anija vald ),11358.83,10589.10,46010.000000,4.0,...,S11002-S11003,8,2020-10-15,1,72245.00,100.00,21947.93,31586.18,7.500000e-01,11502.50
134473,16076821,PROFFROOF.EE OÜ,Äriühing,jah,EHITUS,Harju ( Tallinn ),244.28,236.60,21132.710000,3.0,...,S11002-S11003,8,2020-10-15,1,31799.38,100.00,480.88,493.40,9.800000e-01,7044.24
134474,16076838,EDIGITO OÜ,Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Harju ( Tallinn ),0.10,0.10,3737.440000,1.0,...,S11002-S11003,8,2020-10-15,1,3752.39,100.00,0.20,0.60,3.737440e+09,3737.44
134475,16076844,DIGITAL NATION DATA OÜ,Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Harju ( Tallinn ),1850.80,0.00,0.000001,1.0,...,S11002-S11003,8,2020-10-15,1,0.00,0.00,1850.80,5552.40,0.000000e+00,0.00


In [171]:
during_cl_df = pd.read_csv("./data/csv/final_csv_df/cl_data/during.csv")
during_cl_df.head()

,Registration_number,Name_x,Type,Registered_for_VAT,Business_area_by_EMTAK_system,County,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,Business_area_by_EMTAK_system_letter,Business_are_by_EMTAK_system_number,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date
0,10015770,"REPTEKS, OÜ",Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Lääne-Viru ( Kadrina vald ),0.00,0.00,5223.25,NaN,2020_iv,M,7112,S11002-S11003,8,1992-06-18,1.0
1,14950082,MARJA EHITUS OÜ,Äriühing,jah,EHITUS,Pärnu ( Pärnu linn ),1829.46,2051.70,3686.00,4.0,2020_iv,F,4391,S11002-S11003,8,2020-04-09,1.0
2,14950076,LIFTEST OMME OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),523.73,0.00,8891.86,NaN,2020_iv,G,4520,S11002-S11003,8,2020-04-09,1.0
3,14950030,IGG TOOLING OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),1820.53,693.03,14569.64,1.0,2020_iv,G,4675,S11002-S11003,8,2020-04-09,1.0
4,14950018,LERTISENTO OÜ,Äriühing,jah,HALDUS- JA ABITEGEVUSED,Harju ( Tallinn ),6282.95,4030.22,21159.22,4.0,2020_iv,N,7820,S11002-S11003,8,2020-04-09,1.0


In [173]:
during_feat_eng = during_cl_df.groupby("Registration_number").apply(feat_eng)
during_feat_eng

,Registration_number,Name_x,Type,Registered_for_VAT,Business_area_by_EMTAK_system,County,All_taxes,Labor_taxes,Turnover,number_of_employees,...,Institutional_classification,Client_identifyer_by_Estonian_Central_Bank,Registration_date,Deletion_date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee
0,10015770,"REPTEKS, OÜ",Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Lääne-Viru ( Kadrina vald ),0.00,0.00,5223.250000,1.0,...,S11002-S11003,8,1992-06-18,1.0,5223.25,37.22,0.00,0.00,0.000000e+00,5223.25
1,14950082,MARJA EHITUS OÜ,Äriühing,jah,EHITUS,Pärnu ( Pärnu linn ),1829.46,2051.70,3686.000000,4.0,...,S11002-S11003,8,2020-04-09,1.0,3686.00,6.43,3881.16,3881.16,0.000000e+00,921.50
2,14950076,LIFTEST OMME OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),523.73,0.00,8891.860000,1.0,...,S11002-S11003,8,2020-04-09,1.0,8891.86,78.92,523.73,523.73,0.000000e+00,8891.86
3,14950030,IGG TOOLING OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),1820.53,693.03,14569.640000,1.0,...,S11002-S11003,8,2020-04-09,1.0,14569.64,16.09,2513.56,2513.56,0.000000e+00,14569.64
4,14950018,LERTISENTO OÜ,Äriühing,jah,HALDUS- JA ABITEGEVUSED,Harju ( Tallinn ),6282.95,4030.22,21159.220000,4.0,...,S11002-S11003,8,2020-04-09,1.0,21159.22,29.86,10313.17,10313.17,0.000000e+00,5289.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31060,14991230,SEMPRE MEGLIO OÜ,Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Harju ( Tallinn ),0.10,0.10,121.160000,1.0,...,S11002-S11003,8,2020-06-16,1.0,10929.96,100.00,0.20,0.60,-8.400000e-01,121.16
31061,14991247,ARSI GARAAZ OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Lääne-Viru ( Rakvere vald ),166.07,0.00,9317.770000,1.0,...,S11002-S11003,8,2020-06-16,1.0,29764.74,100.00,166.07,818.62,4.000000e-02,9317.77
31062,14991276,REVFRESH MARKETPLACE OÜ,Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Lääne-Viru ( Rakvere linn ),440.00,0.00,2250.000000,1.0,...,S11002-S11003,8,2020-06-16,1.0,2838.78,100.00,440.00,1760.00,2.250000e+09,2250.00
31063,14991313,ESTONIAN NATURAL HONEY OÜ,Äriühing,jah,"PÕLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK",Pärnu ( Pärnu linn ),0.00,0.00,4993.910000,1.0,...,S11002-S11003,8,2020-06-16,1.0,50086.66,100.00,0.00,340.65,0.000000e+00,4993.91


# for labelling

In [179]:
from numpy import log as ln
from numpy import inf

def get_int_quarter(q):
    if q =="i":
        return 1
    if q=="ii":
        return 2
    if q=="iii":
        return 3
    if q=="iv":
        return 4

In [221]:
def add_label_feat(g):
    ex_df = pd.DataFrame(g).reset_index()
    #print(ex_df)
    E_beg = list(ex_df['number_of_employees'])[0]
    E_end = list(ex_df['number_of_employees'])[-1]
    try:
        Birch_index = (E_end-E_end)*(E_end/E_beg)
    except:
        Birch_index = 0
    ex_df['Birch_index'] = Birch_index
    
    #print(ex_df['year_quarter'])
    
    st_year = int(list(ex_df['year_quarter'])[0].split("_")[0])
    st_year
    
    #print(list(ex_df['year_quarter']))
    st_quarter = list(ex_df['year_quarter'])[0].split("_")[1]
    st_quarter = get_int_quarter(st_quarter)

    en_year = int(list(ex_df['year_quarter'])[-1].split("_")[0])
    en_year


    en_quarter = list(ex_df['year_quarter'])[-1].split("_")[1]
    en_quarter = get_int_quarter(en_quarter)
    en_quarter

    dur_q =(4 - (st_quarter - 1)) + (((en_year-st_year) - 1) *4) + en_quarter
    

    # employment_growth_rate
    emp_growth_rate_CGR = (ln(E_end) - ln(E_beg))/dur_q
    ex_df['emp_growth_rate_CGR'] = emp_growth_rate_CGR
    
    # absolute change in the number of employees
    ACE = abs(E_end - E_beg)
    ex_df['ACE'] = ACE
    

    return ex_df




In [222]:
all_label_feat = all_feat_eng.groupby("Registration_number").apply(add_label_feat).reset_index(drop=True)
all_label_feat

,index,Registration_number,Name_x,Type,Registered_for_VAT,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,...,Deletion_date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,Birch_index,emp_growth_rate_CGR,ACE
0,47394,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,Harju maakond,INFO JA SIDE,12126.74,11533.92,51486.75,...,1,51486.75,100.00,23660.66,23660.66,0.00,5720.75,0.0,0.000000,0.0
1,47398,10000127,"ARAVETE APTEEK, TÜ",Äriühing,jah,Järva maakond,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,6595.39,3983.36,106263.62,...,1,106263.62,100.00,10578.75,10578.75,0.00,53131.81,0.0,0.000000,0.0
2,47387,10000165,"KIVIÕLI KAUBAHOOV, AS",Äriühing,jah,Ida-Viru maakond,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,64705.68,39278.43,501856.24,...,1,501856.24,100.00,103984.11,103984.11,0.00,15207.76,0.0,0.000000,0.0
3,47372,10000219,"YEAR, AS",Äriühing,jah,Harju maakond,TÖÖTLEV TÖÖSTUS,21555.48,23840.41,90645.11,...,1,90645.11,100.00,45395.89,45395.89,0.00,2666.03,0.0,0.000000,0.0
4,47379,10000248,ITW CONSTRUCTION PRODUCTS OÜ,Äriühing,jah,Harju maakond,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,92882.60,22990.97,888253.22,...,1,888253.22,100.00,115873.57,115873.57,0.00,177650.64,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281776,266543,16127903,EIPPA WSOPERATIONS OÜ,Äriühing,jah,Harju ( Tallinn ),HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,71488.78,76192.07,1221324.36,...,1,1222484.81,58.39,147680.85,160115.44,1051.46,5333.29,0.0,0.547249,279.0
281777,273624,16127903,EIPPA WSOPERATIONS OÜ,Äriühing,jah,Harju ( Tallinn ),HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,261333.19,277445.91,871215.43,...,1,2093700.24,100.00,538779.10,698894.54,-0.29,2517.96,0.0,0.547249,279.0
281778,272273,16196755,KMG OÜ,Äriühing,jah,Harju ( Tallinn ),EHITUS,315204.79,273426.19,5437446.59,...,1,5437446.59,23.92,588630.98,588630.98,0.00,27601.25,0.0,-0.002545,1.0
281779,274303,16196755,KMG OÜ,Äriühing,jah,Harju ( Tallinn ),EHITUS,1084980.06,705252.97,17291141.71,...,1,22728588.30,100.00,1790233.03,2378864.01,2.18,88220.11,0.0,-0.002545,1.0


In [223]:
all_label_feat = all_label_feat.drop(["index"], 1).reset_index(drop=True)
all_label_feat

,Registration_number,Name_x,Type,Registered_for_VAT,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,...,Deletion_date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,Birch_index,emp_growth_rate_CGR,ACE
0,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,Harju maakond,INFO JA SIDE,12126.74,11533.92,51486.75,9.0,...,1,51486.75,100.00,23660.66,23660.66,0.00,5720.75,0.0,0.000000,0.0
1,10000127,"ARAVETE APTEEK, TÜ",Äriühing,jah,Järva maakond,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,6595.39,3983.36,106263.62,2.0,...,1,106263.62,100.00,10578.75,10578.75,0.00,53131.81,0.0,0.000000,0.0
2,10000165,"KIVIÕLI KAUBAHOOV, AS",Äriühing,jah,Ida-Viru maakond,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,64705.68,39278.43,501856.24,33.0,...,1,501856.24,100.00,103984.11,103984.11,0.00,15207.76,0.0,0.000000,0.0
3,10000219,"YEAR, AS",Äriühing,jah,Harju maakond,TÖÖTLEV TÖÖSTUS,21555.48,23840.41,90645.11,34.0,...,1,90645.11,100.00,45395.89,45395.89,0.00,2666.03,0.0,0.000000,0.0
4,10000248,ITW CONSTRUCTION PRODUCTS OÜ,Äriühing,jah,Harju maakond,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,92882.60,22990.97,888253.22,5.0,...,1,888253.22,100.00,115873.57,115873.57,0.00,177650.64,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281776,16127903,EIPPA WSOPERATIONS OÜ,Äriühing,jah,Harju ( Tallinn ),HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,71488.78,76192.07,1221324.36,229.0,...,1,1222484.81,58.39,147680.85,160115.44,1051.46,5333.29,0.0,0.547249,279.0
281777,16127903,EIPPA WSOPERATIONS OÜ,Äriühing,jah,Harju ( Tallinn ),HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,261333.19,277445.91,871215.43,346.0,...,1,2093700.24,100.00,538779.10,698894.54,-0.29,2517.96,0.0,0.547249,279.0
281778,16196755,KMG OÜ,Äriühing,jah,Harju ( Tallinn ),EHITUS,315204.79,273426.19,5437446.59,197.0,...,1,5437446.59,23.92,588630.98,588630.98,0.00,27601.25,0.0,-0.002545,1.0
281779,16196755,KMG OÜ,Äriühing,jah,Harju ( Tallinn ),EHITUS,1084980.06,705252.97,17291141.71,196.0,...,1,22728588.30,100.00,1790233.03,2378864.01,2.18,88220.11,0.0,-0.002545,1.0


In [226]:
Before_label_feat = before_feat_eng.groupby("Registration_number").apply(add_label_feat).reset_index(drop=True)
Before_label_feat = Before_label_feat.drop(["index"], 1).reset_index(drop=True)

Before_label_feat

,Registration_number,Name_x,Type,Registered_for_VAT,Business_area_by_EMTAK_system,County,All_taxes,Labor_taxes,Turnover,number_of_employees,...,Deletion_date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,Birch_index,emp_growth_rate_CGR,ACE
0,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,INFO JA SIDE,Harju maakond,14051.48,11547.12,91352.65,8.0,...,1,91352.65,100.00,25598.60,25598.60,0.00,11419.08,0.0,0.000000,0.0
1,10000127,"ARAVETE APTEEK, TÜ",Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,Järva maakond,6004.08,3230.05,130599.87,2.0,...,1,130599.87,100.00,9234.13,9234.13,0.00,65299.94,0.0,0.000000,0.0
2,10000165,"KIVIÕLI KAUBAHOOV, AS",Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,Ida-Viru maakond,82240.16,39572.41,664117.39,35.0,...,1,664117.39,100.00,121812.57,121812.57,0.00,18974.78,0.0,0.000000,0.0
3,10000219,"YEAR, AS",Äriühing,jah,TÖÖTLEV TÖÖSTUS,Harju maakond,18114.89,17804.11,81179.30,25.0,...,1,81179.30,100.00,35919.00,35919.00,0.00,3247.17,0.0,0.000000,0.0
4,10000248,ITW CONSTRUCTION PRODUCTS OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOO...,Harju maakond,146758.38,25706.93,1251151.81,6.0,...,1,1251151.81,100.00,172465.31,172465.31,0.00,208525.30,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134472,16127903,EIPPA WSOPERATIONS OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),71488.78,76192.07,1221324.36,229.0,...,1,1221324.36,58.37,147680.85,147680.85,0.00,5333.29,0.0,0.206358,117.0
134473,16127903,EIPPA WSOPERATIONS OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),261333.19,277445.91,871215.43,346.0,...,1,2092539.79,100.00,538779.10,686459.95,-0.29,2517.96,0.0,0.206358,117.0
134474,16196755,KMG OÜ,Äriühing,jah,EHITUS,Harju ( Tallinn ),315204.79,273426.19,5437446.59,197.0,...,1,5437446.59,23.92,588630.98,588630.98,0.00,27601.25,0.0,-0.002545,1.0
134475,16196755,KMG OÜ,Äriühing,jah,EHITUS,Harju ( Tallinn ),1084980.06,705252.97,17291141.71,196.0,...,1,22728588.30,100.00,1790233.03,2378864.01,2.18,88220.11,0.0,-0.002545,1.0


In [229]:
During_label_feat = during_feat_eng.groupby("Registration_number").apply(add_label_feat).reset_index(drop=True)
During_label_feat = During_label_feat.drop(["index"], 1).reset_index(drop=True)

During_label_feat

,index,Registration_number,Name_x,Type,Registered_for_VAT,Business_area_by_EMTAK_system,County,All_taxes,Labor_taxes,Turnover,...,Deletion_date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,Birch_index,emp_growth_rate_CGR,ACE
0,0,10015770,"REPTEKS, OÜ",Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Lääne-Viru ( Kadrina vald ),0.00,0.00,5.223250e+03,...,1.0,5223.25,37.22,0.00,0.00,0.00,5223.25,0.0,0.000000,0.0
1,10186,10015770,"REPTEKS, OÜ",Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Lääne-Viru ( Kadrina vald ),205.50,0.00,2.058170e+03,...,1.0,7281.42,51.88,205.50,205.50,-0.61,2058.17,0.0,0.000000,0.0
2,16097,10015770,"REPTEKS, OÜ",Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Lääne-Viru ( Kadrina vald ),68.50,0.00,1.844470e+03,...,1.0,9125.89,65.02,68.50,274.00,-0.10,1844.47,0.0,0.000000,0.0
3,26231,10015770,"REPTEKS, OÜ",Äriühing,jah,"KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS",Lääne-Viru ( Kadrina vald ),0.00,0.00,4.909420e+03,...,1.0,14035.31,100.00,0.00,274.00,1.66,4909.42,0.0,0.000000,0.0
4,6941,10041649,LNG SERVICE OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),0.00,0.00,1.000000e-06,...,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31060,16088,16127903,EIPPA WSOPERATIONS OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),71488.78,76192.07,1.221324e+06,...,1.0,1222484.81,58.39,147680.85,160115.44,1051.46,5333.29,0.0,0.547249,279.0
31061,23162,16127903,EIPPA WSOPERATIONS OÜ,Äriühing,jah,HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,Harju ( Tallinn ),261333.19,277445.91,8.712154e+05,...,1.0,2093700.24,100.00,538779.10,698894.54,-0.29,2517.96,0.0,0.547249,279.0
31062,17778,16196755,KMG OÜ,Äriühing,jah,EHITUS,Harju ( Tallinn ),315204.79,273426.19,5.437447e+06,...,1.0,5437446.59,23.92,588630.98,588630.98,0.00,27601.25,0.0,-0.002545,1.0
31063,23714,16196755,KMG OÜ,Äriühing,jah,EHITUS,Harju ( Tallinn ),1084980.06,705252.97,1.729114e+07,...,1.0,22728588.30,100.00,1790233.03,2378864.01,2.18,88220.11,0.0,-0.002545,1.0


In [239]:
all_label_feat.to_csv( "./data/csv/final_csv_df/feat_eng_label/all.csv", index=False)
Before_label_feat.to_csv( "./data/csv/final_csv_df/feat_eng_label/before.csv", index=False)
During_label_feat.to_csv( "./data/csv/final_csv_df/feat_eng_label/during.csv", index=False)